In [1]:
'''

Lesson 2: DeepLearningPytorch

Use pre-trained networks to solved challenging problems in computer vision
example: use networks trained on ImageNet available from torchvision.

transfer learning = using pre-trained network on images NOT in the training set

goal:  train a network that can classify our cat and dog photos with near perfect accuracy

'''

# download pre-trained networks with torchvision

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch.autograd as autograd
from torch.autograd import Variable

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [2]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
# means = [0.485, 0.456, 0.406] - standard deviations = [0.229, 0.224, 0.225].
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
# SKIP load in model like densenet
# https://pytorch.org/docs/0.3.0/torchvision/models.html#id5
# model has features + classifiers

model = models.densenet121(pretrained=True)
model

In [ ]:
'''
SKIP

Model is built out of two main parts, the features and the classifier. 
The features part is a stack of convolutional layers and overall works as a feature detector 
that can be fed into a classifier. 

The classifier part is a single fully-connected layer (classifier): 
Linear(in_features=1024, out_features=1000). 
This layer was trained on the ImageNet dataset, so it won't work for our specific problem. 
That means we need to replace the classifier, but the features will work perfectly on their own.
'''
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [ ]:
# SKIP

# model built - train classifier
# using really deep neural network
# need to use GPU to do calculations - linear algebra calculations in parallel
# pytorch uses CUDA compare forward and backward passes on GPU = model.to('cuda')
# move back to CPU = model.to('cpu')

#model.cuda()
#images.cuda()
model.cpu(), images.cpu()
# result: Found no NVIDIA driver on your system

In [ ]:
# SKIP import time

In [ ]:
# SKIP 

for cuda in [False, True]:

# for device in ['cpu', 'cuda']:
    
    criterion = nn.NLLLoss()
    # Only train classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
    
    if cuda:
       model.cuda()
    else:
       model.cpu()
    # model.to(device)
        
    for ii, (inputs, labels) in enumerate(trainloader):
        inputs, labels = Variable(inputs), Variable(labels)
        #inputs, labels = inputs.to(device), labels.to(device)
        
        if cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
            
        start = time.time()
        
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if ii==3:
            break
 
        # print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")
    print(f"CUDA = {cuda}; Time per batch: {(time.time() - start)/3:.3f} seconds")

In [ ]:
#cuda = torch.cuda.is_available()

In [3]:
## TODO: Use a pretrained model to classify the cat and dog images
# Use GPU if it's available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#choose pretrained models

#model = models.densenet121(pretrained=True)
model = models.resnet50(pretrained=True)

In [5]:
#look at end for fc when using model.resnet50
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [6]:
# Freeze parameters so we don't backprop through them

# turn off gradiants for model
for param in model.parameters():
    param.requires_grad = False

# define new classifier
model.classifier = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 2),
                                 nn.LogSoftmax(dim=1))

#if using model.resnet50
model.fc = model.classifier

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device);
print(device)

cpu


In [7]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
    for images, labels in trainloader:
        steps += 1
        
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        #lo probabilities
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            test_loss = 0
            accuracy = 0
            
            for images, labels in testloader:
                
                # transfer tensors to GPU
                images, labels = images.to(device), labels.to(device)
                
                logps = model(images)
                loss = criterion(logps, labels)
                test_loss += loss.item()
        
                # calculate our accuracy
                ps = torch.exp(logps)
                top_ps, top_class = ps.topk(1, dim=1)
                equality = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equality.type(torch.FloatTensor))
                
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            
            running_loss = 0
            model.train()

KeyboardInterrupt: 